<a href="https://colab.research.google.com/github/Jayathilaga/Emotiona_sarcasm_detection/blob/main/Emotion_Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers==4.0.1 sentencepiece==0.1.94 torch torchvision flask flair

     |████████████████████████████████| 1.4MB 9.3MB/s 
     |████████████████████████████████| 1.1MB 38.6MB/s 
     |████████████████████████████████| 286kB 53.8MB/s 
     |████████████████████████████████| 901kB 50.7MB/s 
     |████████████████████████████████| 2.9MB 40.2MB/s 
     |████████████████████████████████| 798kB 47.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 19.7MB 156kB/s 
     |████████████████████████████████| 983kB 34.3MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9705 sha256=bca13e9cc2fe45b83d06e88edf3e6d244aa70e0472c12960ce6dbfeb5c7bd846
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for mpld3: filename=mpld3-0.3-cp37-none-any.whl size=116704 sha256=0de8d62c2

In [ ]:
'''
import flask
app = flask.Flask(__name__)
app.config["DEBUG"] = True
'''

'\nimport flask\napp = flask.Flask(__name__)\napp.config["DEBUG"] = True\n'

In [ ]:
import transformers
# import protobuf
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tokenizers import Tokenizer
from textblob import TextBlob
import re
from flask import jsonify
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import flair
import random

ImportError: ignored

In [ ]:
flair_sentiment = flair.models.TextClassifier.load('sentiment-fast')

2021-02-10 18:29:58,291 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-fasttext-rnn/sentiment-en-mix-ft-rnn.pt not found in cache, downloading to /tmp/tmpbl5uto_p


100%|██████████| 1242007042/1242007042 [00:49<00:00, 25250639.16B/s]

2021-02-10 18:30:47,829 copying /tmp/tmpbl5uto_p to cache at /root/.flair/models/sentiment-en-mix-ft-rnn.pt


2021-02-10 18:30:52,488 removing temp file /tmp/tmpbl5uto_p
2021-02-10 18:30:52,632 loading file /root/.flair/models/sentiment-en-mix-ft-rnn.pt


In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# Emotion Recognition

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion", use_fast=True)

model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-emotion")

Some weights of the model checkpoint at mrm8488/t5-base-finetuned-emotion were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Sarcasm Detection

tokenizer2 = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter", use_fast=True)

model2 = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")

Some weights of the model checkpoint at mrm8488/t5-base-finetuned-sarcasm-twitter were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_emotion(text):

  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')
  # input_ids = tokenizer(text + '</s>',  return_tensors='pt').input_ids
  # print(input_ids.tokens)

  output = model.generate(input_ids=input_ids,
               max_length=2)

  dec = [tokenizer.decode(ids) for ids in output]
  label = RemoveModelTags(dec[0].strip())
  #print(text)
  #print("Emotion: ", label[6:])
  #print()
  return label

In [ ]:
def RemoveModelTags(text): 
      
    # Print string after removing tags 
    return re.compile(r'<[^>]+>').sub('', text).strip()

In [ ]:
def check_sarcasm(text):

  input_ids = tokenizer2.encode(text + '</s>', return_tensors='pt')
  
  output = model2.generate(input_ids=input_ids, max_length=3)

  dec = [tokenizer2.decode(ids) for ids in output]
  label = RemoveModelTags(dec[0].strip())
  
  if label=="derison":
    return "Yes"
  elif label=="normal":
    return "No"

In [ ]:
def get_eval_emotion(text):
    if check_sarcasm(text) == 'Yes':
        return 'sarcasm'
    elif get_sentiment(text) == 'neutral':
        return "neutral"
    elif get_sentiment(text) == 'negative':
        return "anger"
    else:
        return get_emotion(text)
    '''elif (get_sentiment(text) == 'Negative' & get_emotion(text)== 'Joy'):
        return get_sentiment(text)'''

In [ ]:
def get_emotion_textblob(text):
    feedback_polarity = TextBlob(text).sentiment.polarity
    if feedback_polarity==0:
      return 'Neutral'
    elif feedback_polarity>0:
      return 'Positive'
    else:
      return 'Negative'

In [ ]:
def get_sentiment(sentence):
    sid = SentimentIntensityAnalyzer()
    if (sid.polarity_scores(sentence)["neu"]==1):
      return "neutral"
    else:
      s = flair.data.Sentence(sentence)
      flair_sentiment.predict(s)
      for label in s.labels:
          value = str(label.value).lower()
      return value

In [ ]:
def get_eval_emotion(text):
    if check_sarcasm(text) == 'Yes':
        return 'sarcasm'
    elif get_sentiment(text) == 'neutral':
        return "neutral"
    elif get_sentiment(text) == 'negative':
        return "anger"
    else:
        return get_emotion(text)
    '''elif (get_sentiment(text) == 'Negative' & get_emotion(text)== 'Joy'):
        return get_sentiment(text)'''

In [ ]:
#@app.route('/api/get_emotion/<text>', methods=['GET'])
def final_result(text):
  print(text)
  #print({'Emotion': get_eval_emotion(text), 'Sentiment': get_emotion_textblob(text)})
  print()
  #response.headers.add("Access-Control-Allow-Origin", "*")
  return ({'emotion': get_eval_emotion(text), 'sentiment': get_sentiment(text), 'domain': "payment_info", 'query': text, 'response': get_response(text)})
  #return ({'Emotion': get_emotion(text), 'Sarcasm': check_sarcasm(text), 'Sentiment': get_emotion_textblob(text), 'Query': text})
#app.run()

In [ ]:
def get_response(text):

  anger = ("We are sorry, I know how frustrating this must be. I'll help your way out of this.", 
    "I know a mistake like this can be very upsetting, especially at this time of year. We'll do everything required to resolve it at the earliest",
    "Apologies for the inconvenience caused! I'll help you fix that asap!")
  fear = ("Don't be scared, We'll help you out.", "I know it can be scary sometimes. Let me help you!", "It isn't as scary as you think. I will help you get out of this!")
  joy = ("I am as excited as you are! I'll help you with that!", "I am happy for you! I'll do the needful!", "Yay! We are excited too! I hope you have a great rest of the year!")
  neutral = ("Sure, I'll help you with that query", "Give us a min, Let me check on this!", "Let me check on this")
  sarcasm = ("I can understand your sarcasm! Sorry about your bad experience with us. Let me help you!", "Hey! Sorry about your experience with this!", "Apologies for the inconvenience! Let me know what can be done!")
  response = get_eval_emotion(text)
  #print("response num", response[num])
  if(response =='anger'):
    return anger[random.randrange(0,3)]
  elif (response =='joy'):
    return joy[random.randrange(0,3)]
  elif (response=='neutral'):
    return neutral[random.randrange(0,3)]
  elif (response=='sarcasm'):
    return sarcasm[random.randrange(0,3)]
  elif (response=='fear'):
    return fear[random.randrange(0,3)]

In [ ]:
#get_emotion("Hey! I got a new phone and would like to be added to my home plan with disney plus")
#get_emotion("My wifi isnt connecting to my phone, i need it now. please help me fix it asap")
#get_emotion("I should have gone with my neighbour's words, your network that has great signal")
#eval_conversation("I should have gone with my neighbour's words, your network that has great signal")
#eval_conversation("so you have a great plan? haha.. that charges me for connecting to a device thats not included in the plan")

In [ ]:
final_result("Hey! I got a new phone with Verizon Network, I need help with the set up")

Hey! I got a new phone with Verizon Network, I need help with the set up



{'domain': 'payment_info',
 'emotion': 'joy',
 'query': 'Hey! I got a new phone with Verizon Network, I need help with the set up',
 'response': "I am as excited as you are! I'll help you with that!",
 'sentiment': 'positive'}

In [ ]:
final_result("I should have gone with my neighbour's words, your network that has great signal")

I should have gone with my neighbour's words, your network that has great signal



{'domain': 'payment_info',
 'emotion': 'sarcasm',
 'query': "I should have gone with my neighbour's words, your network that has great signal",
 'response': 'Apologies for the inconvenience! Let me know what can be done!',
 'sentiment': 'positive'}

In [ ]:
final_result("My wifi isnt connecting to my phone, i need it now. please help me fix it asap, how much time should i wait?")

My wifi isnt connecting to my phone, i need it now. please help me fix it asap, how much time should i wait?



{'domain': 'payment_info',
 'emotion': 'anger',
 'query': 'My wifi isnt connecting to my phone, i need it now. please help me fix it asap, how much time should i wait?',
 'response': "Apologies for the inconvenience caused! I'll help you fix that asap!",
 'sentiment': 'negative'}

In [ ]:
final_result("I need help with connecting to network can you let me know the steps")

I need help with connecting to network can you let me know the steps



{'domain': 'payment_info',
 'emotion': 'fear',
 'query': 'I need help with connecting to network can you let me know the steps',
 'response': 'I know it can be scary sometimes. Let me help you!',
 'sentiment': 'positive'}

In [ ]:
final_result("My phone doesn't have signal now. could you please check asap")

My phone doesn't have signal now. could you please check asap



{'domain': 'payment_info',
 'emotion': 'anger',
 'query': "My phone doesn't have signal now. could you please check asap",
 'response': "Apologies for the inconvenience caused! I'll help you fix that asap!",
 'sentiment': 'negative'}

In [ ]:
final_result("show my profile info")

show my profile info



{'domain': 'payment_info',
 'emotion': 'neutral',
 'query': 'show my profile info',
 'response': 'Let me check on this',
 'sentiment': 'neutral'}

In [ ]:
final_result("I feel low. Your network isnt helping me with connectivity")

I feel low. Your network isnt helping me with connectivity



{'domain': 'payment_info',
 'emotion': 'anger',
 'query': 'I feel low. Your network isnt helping me with connectivity',
 'response': "We are sorry, I know how frustrating this must be. I'll help your way out of this.",
 'sentiment': 'negative'}

In [ ]:
!## Save Emotion Recognition Model
model.save_pretrained('./my_saved_model_directory/')
tokenizer.save_pretrained('./my_saved_model_directory/')

In [ ]:
## Reload Emotion Recognition Model
model = AutoModelForSeq2SeqLM.from_pretrained('./my_saved_model_directory/')
tokenizer = AutoTokenizer.from_pretrained('./my_saved_model_directory/', use_fast=True)

In [ ]:
## Save Sarcasm Detection Model
model2.save_pretrained('./my_saved_model_directory/')
tokenizer2.save_pretrained('./my_saved_model_directory/')

In [ ]:
# Reload Sarcasm Detection Model
model2 = AutoModelForSeq2SeqLM.from_pretrained('./my_saved_model_directory/')
tokenizer2 = AutoTokenizer.from_pretrained('./my_saved_model_directory/', use_fast=True)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
! git clone https://github.com/huggingface/torchMoji.git

In [ ]:
! git pull

In [ ]:
import pickle
import re
from nltk.stem import WordNetLemmatizer
def getIntent(text):
  with open('text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)
  stemmer = WordNetLemmatizer()
  document = re.sub(r'\W', ' ', str(text))    
  document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
  document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
  document = re.sub(r'\s+', ' ', document, flags=re.I)
  document = re.sub(r'^b\s+', '', document)
  document = document.lower()
  document = document.split()
  document = [stemmer.lemmatize(word) for word in document]
  document = ' '.join(document)
  with open('tfidf_converter', 'rb') as picklefile:
    tfidfconverter = pickle.load(picklefile)
  X = tfidfconverter.transform([document]).toarray()
  y = model.predict(X)
  return y

In [ ]:
#train_df = pd.read_csv('https://www.dropbox.com/s/1pzkadrvffbqw6o/train.txt', sep=';')
#with client.get_file('https://www.dropbox.com/s/1pzkadrvffbqw6o/train.txt') as f:
df_train = pd.read_csv('/content/gdrive/MyDrive/Datasets/train.txt', delimiter=';', header=None, names=['sentence','label'])
#train_df.columns = ['sentence', 'emotion']
df_train[:20]

In [ ]:
df_train = pd.read_csv('/content/gdrive/MyDrive/Datasets/goemotions_1.csv', header=0, usecols=['text','neutral'])
df_neutral = df_train[df_train['neutral']==1]
df_neutral.head()

# import numpy as np
# df_neutral_text.to_csv('/content/gdrive/MyDrive/Datasets/neutral.csv', header=False, index=False)
# np.savetxt(r'/content/gdrive/MyDrive/Datasets/neutral.txt', df_neutral, fmt=['%d','%d'], delimiter='\t')

In [ ]:
import string
import re
import os
import nltk
nltk.download('twitter_samples')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples
import pandas as pd

tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

# Stop words are messy and not that compelling; 
# "very" and "not" are considered stop words, but they are obviously expressing sentiment

# The porter stemmer lemmatizes "was" to "wa".  Seriously???

# I'm not sure we want to get into stop words
stopwords_english = stopwords.words('english')

# Also have my doubts about stemming...
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def preprocess_text(query):
    '''
    Input: 
        query: a string containing a query
    Output:
        querys_clean: a list of words containing the processed query
    
    '''
    # remove stock market tickers like $GE
    query = re.sub(r'\$\w*', '', query)
    # remove old style requery text "RT"
    query = re.sub(r'^RT[\s]+', '', query)
    # remove hyperlinks
    query = re.sub(r'https?:\/\/.*[\r\n]*', '', query)
    query = re.sub('@[\w]+','',query)
    # remove hashtags
    # only removing the hash # sign from the word
    query = re.sub(r'#', '', query)
    # tokenize querys
    #tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    #query_tokens = tokenizer.tokenize(query)
    ### START CODE HERE ###
    #querys_clean = []
    '''
    for word in query_tokens:
        if (word not in stopwords_english and # remove stopwords
            word not in string.punctuation): # remove punctuation
            #querys_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            querys_clean.append(stem_word)
    ### END CODE HERE ###
    '''
    return query

In [ ]:
df = pd.read_csv('https://query.data.world/s/i2s6zk24kbc3drc5vbyjocsnrafc7g', header=0, usecols=['sentiment','content'])
#df2 = df[(df.a != -1) | (df.b != -1)]
df_neutral_empty_data = df[(df['sentiment']=='empty') | (df['sentiment']=='neutral')]
#df_neutral_data = df[(df['sentiment']=='neutral')]
#len(df_neutral_data)
df_neutral_empty_data_content = df_neutral_empty_data['content']
#df_neutral_empty_data_content.head()
df_neutral_empty_data_content.loc['content'] = df_neutral_empty_data_content.apply(lambda x: preprocess_text(x))

df_neutral_empty_data_content.loc['content'].to_csv('neutral.csv', header=False, index=False)